# Student Name: Zhongyue Xing

# Preface

Antiviral drugs are used for treating viral infections. The procedure often requires a sequence of treatments. Over time patients develop resistance to the drugs and need to keep switching them. Moreover, a drug of type 1 can make the patient (partially) resistant to the drug of type 2.

In this problem we consider drugs of type A, B, and C, all of which can potentially lower the viral load but the effect depends on their order. The 'Viral_load.xlsx' dataset provides results for 1,000 of patients, where rewards (based on the observed viral load level of patients) indicate the effectiveness of the preceding drugs used. For example, the first patient used drugs A, B, and C in this order and received the following rewards: 14.9062033, 13.67860579, and 12.76096513, respectively.

Because of the antiviral drug resistance, same drug cannot be used twice. The policy (behavior policy $b$) that was used for treatments represented in the 'Viral_load.xlsx' dataset is

1) if all drugs are available, pick A with probability 0.7, B with probability 0.2, and C with probability 0.1   
2) if all drugs, but A, are available, pick B with probability 0.65 and C with probability 0.35   
3) if all drugs, but B, are available, pick A with probability 0.9 and C with probability 0.1  
4) if all drugs, but C, are available, pick A with probability 0.8 and B with probability 0.2   
5) if only one drug is available, pick it with probability 1    



One can find that based on these observations, the optimal order of the treatment is B, A, and then C, indicating that drug A diminishes the effect of drug B, and C diminishes the effect of drug A (and possibly B).

The sequence of treatments can be represented as a Markov Decision Process (MDP). Clearly, the set of available actions at time t depends on the intire history. In order to model the treatment proceedure with the MDP, we introduce states as follows:

1) $S_{ABC}=${all drugs are available}   
2) $S_{BC}=${all drugs, except A, are available}   
3) $S_{AC}=${all drugs, except B, are available}       
4) $S_{AB}=${all drugs, except C, are available}   
5) $S_{A}=${only A is available}   
6) $S_{B}=${only B is available}   
7) $S_{C}=${only C is available}      

The optimal policy $\pi$, that corresponds to the sequence B, A, C, is then: (1) select B in state $S_{ABC}$; (2) select A in state $S_{AC}$; (3) select C in state $S_{C}$. The other actions do not need to be specified because the MDP will never reach them.
  

## Problem 1 (5 points)

In this problem, create a function that returns the importance-sampling ratio for each sequence of type "ABC" (use input format of choice), given target and behavior policies, $\pi$ and b.

In [1]:
b = {
    'ABC': {'A': 0.7, 'B': 0.2, 'C': 0.1},
    'BC': {'B': 0.65, 'C': 0.35},
    'AC': {'A': 0.9, 'C': 0.1},
    'AB': {'A': 0.8, 'B': 0.2},
    'A': {'A': 1},
    'B': {'B': 1},
    'C': {'C': 1}
    }

pi = {"ABC": 'B',
     "AC": 'A',
     "C": 'C'}

def rho(S, A, b = b, pi = pi):
    """
    Returns the importance sampling ratio for optimistic policy pi and behavioral policy b given
    a sequence of states and actions in lists.
    S = [S_t, S_(t+1), ...., S_(T-1), S_T]
    A = [A_t, A_(t+1), ...., A_(T-1), A_T]
    Pi should be an admissible, non-empty, and deterministic policy.
    
    """
    assert len(S) == len(A)

    r = 1
    for k in range(len(S)):
        if S[k] in pi.keys():
            if A[k] == pi[S[k]]:
                r *= 1/b[S[k]][A[k]]
            else:
                return 0
        else:
            return 0

    return r

rho(["ABC", "AC", "C"], ["B", "A", "C"])

5.555555555555555

## Problem 2 (10 points)

Using 'Viral_load.xlsx' dataset, estimate the state value function $v_\pi(s)$ for $s\in\{s_{ABC},s_{AC},s_{C}\}$ via Off-policy MC prediction (section 5.6 of "Reinforcement Learning" by Sutton and Barto). Please notice that generating an episode under policy $b$ would correspond to reading a row from 'Viral_load.xlsx'. Print the result for $\gamma=0.99$.

In [2]:
import csv
import copy

def MC_state_value_estimate(gamma = 0.99):
    V = {state:0 for state in ["ABC", "AC", "C"]}
    C = copy.copy(V)
    
    csv_file = open("Viral_load.csv") # file closed upon return statement
    reader = csv.reader(csv_file)
    next(reader) # skip header row

    while True:
        # Get an episode
        try:
            episode = next(reader)[1:]
            states = ["ABC"]
            actions = []
            rewards = []
            for t in range(len(states[0])):
                actions.append(episode[t*2])
                rewards.append(float(episode[t*2+1]))
                new_state = states[t].replace(actions[t], '')
                if new_state:
                    states.append(new_state)

        # reached final episode
        except StopIteration:
            csv_file.close()
            return V

        G = 0
        W = 1
        for t in range(-1, -len(states)-1, -1):
            if W == 0:
                break

            # get state, action and reward at t
            s = states[t]
            if s not in V.keys():
                break
            a = actions[t]
            r = rewards[t]

            # update relevant values
            G = gamma*G + r
            C[s] += W
            V[s] += W/C[s] * (G-V[s])
            W *= rho([s], [a])


print("The corresponding state value following optimal policy are:")
print(MC_state_value_estimate())

The corresponding state value following optimal policy are:
{'ABC': 41.507588616523094, 'AC': 27.8264130004963, 'C': 14.025463818322676}


## Problem 3 (10 points)
Modify the algorithm you developed in Problem 2 in order to estimate $v_\pi(s)$ for $s\in\{s_{ABC},s_{AC},s_{C}\}$ via Off-policy one-step TD prediction. Print the result for $\gamma=0.99$ and $\alpha=0.15$.

In [3]:
def TD_state_value_estimate(gamma = 0.99, alpha = 0.15):
    V = {state:0 for state in ["ABC", "AC", "C",'']}
    
    csv_file = open("Viral_load.csv") # file closed upon return statement
    reader = csv.reader(csv_file)
    next(reader) # skip header row

    while True:
        # Get an episode
        try:
            episode = next(reader)[1:]

        # terminate after final episode
        except StopIteration:
            csv_file.close()
            del V[''] # remove terminal state place-holder
            return V

        G = 0
        s = "ABC"
        for t in range(len(episode)//2):
            a = episode[t*2]
            if a != pi[s]:
                break
            s_new = s.replace(a, '')
            r = float(episode[t*2+1])

            V[s] += alpha*rho([s], [a])*(r + gamma*V[s_new] - V[s])

            s = s_new


print("The corresponding state value following optimal policy are:")
print(TD_state_value_estimate())

The corresponding state value following optimal policy are:
{'ABC': 40.89128819163807, 'AC': 27.668808615308315, 'C': 14.073557148762944}


In [4]:
exit